In [54]:
import pandas as pd;
import numpy as np;

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score ,accuracy_score

In [56]:
df = pd.read_csv("../Data/preprocessed_data")

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(["num"],axis=1),df["num"],test_size=0.3,random_state =0)

In [58]:
X_train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,exang,oldpeak,slope,ca,thal
168,70,1,3,160,269,0,0,1,2.9,2,1.0,7.0
175,56,1,4,132,184,0,2,1,2.1,2,1.0,6.0
54,60,1,4,130,253,0,0,1,1.4,1,1.0,7.0
135,70,1,4,145,174,0,0,1,2.6,3,0.0,7.0
18,48,0,3,130,275,0,0,0,0.2,1,0.0,3.0


In [59]:
X_train.shape

(207, 12)

In [60]:
# Onehot encoding
trf1 = ColumnTransformer(transformers=[('nominal_cat',OneHotEncoder(sparse=False,handle_unknown='ignore'),[2,5,6,7,9,10,11])],remainder='passthrough')


In [61]:
# Scaling
trf2 = ColumnTransformer(transformers=[('scale',RobustScaler(),slice(0,15))])


In [62]:
#pipeline
pipe = Pipeline([('trf1',trf1),('trf2',trf2)])

In [64]:
X_train_processed = pipe.fit_transform(X_train)
X_test_processed = pipe.transform(X_test)

c:\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [65]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=None, random_state=42)
clf.fit(X_train_processed,y_train)
y_pred = clf.predict(X_test_processed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5777777777777777


In [66]:
scores = cross_val_score(clf, X_train_processed, y_train, cv=5, scoring='accuracy')
print("Cross-validated accuracy:", scores.mean())

Cross-validated accuracy: 0.521602787456446


In [67]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.70      0.94      0.80        48
           1       0.28      0.29      0.29        17
           2       0.25      0.10      0.14        10
           3       0.50      0.10      0.17        10
           4       0.00      0.00      0.00         5

    accuracy                           0.58        90
   macro avg       0.35      0.29      0.28        90
weighted avg       0.51      0.58      0.52        90



In [68]:
import pickle 
pickle.dump(pipe,open('pipe.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))